## SQL Transacties

Door transacties te gebruiken kun je SQL queries "bundelen". Als een query faalt, kun je alle queries terugdraaien en de database in originele staat terugbrengen.

In [ ]:
import os
import sqlite3

In [ ]:
# Maak connectie en cursor
database_file = "sqlite_transacties.db"
connection = sqlite3.connect(database_file)
cursor = connection.cursor()

In [ ]:
# Maak 2 tabellen, merk op dat productnaam UNIQUE moet zijn.
cursor.executescript("""
    CREATE TABLE Producten (
        ProductId INTEGER PRIMARY KEY,
        Naam TEXT UNIQUE,
        Merk TEXT,
        Omschrijving TEXT
    );

    CREATE TABLE ProductKenmerken (
        ProductId INTEGER,
        Kenmerk TEXT,
        Waarde TEXT
    );

""")

In [ ]:
# Voeg een aantal producten en kenmerken tegelijk toe
try:
    cursor.execute("BEGIN TRANSACTION")
    
    # Voeg productkenmerken toe
    cursor.execute("""
        INSERT INTO ProductKenmerken (ProductId, Kenmerk, Waarde)
        VALUES
            (1, 'Test Property A', 1),
            (1, 'Test Property B', 2),
            (1, 'Test Property C', 3)
    """)

    # Voeg product toe
    cursor.execute("""
        INSERT INTO Producten (Naam, Merk, Omschrijving)
            VALUES('Test Product', 'Test Merk', 'Omschrijving van het testproduct.');
    """)

    # Leg de wijzigingen vast
    cursor.execute("COMMIT")
    print("All done!")

except sqlite3.IntegrityError:
    
    print("On nee! Een dubbele product naam!")
    cursor.execute("ROLLBACK")

In [ ]:
cursor.execute("SELECT * FROM Producten;").fetchall()

In [ ]:
cursor.execute("SELECT * FROM ProductKenmerken;").fetchall()

In [ ]:
# Afsluiten en opruimen
try:
    connection.close()
    os.remove(database_file)
except FileNotFoundError:
    pass
except PermissionError:
    print(f"Fout: De database {database_file!r} is nog open!")